In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import itertools
from pprint import pprint
from scipy.special import softmax

In [ ]:
# fix random seed for reproducibility
np.random.seed(0)

In [ ]:
## large alpha => uniformly likely (heterogeneous)
## small alpha => more likely to repeat tasks (identical)


# Initialize
# alpha = 100
# alpha = 10  # Example value for alpha => 0.143
# alpha = 1.0 # Example value for alpha => 0.164
alpha = 0.1 # Example value for alpha => 0.304
# alpha = 0.001


## MNIST
num_classes = 10 # Example value for the number of classes
num_classes_per_task = 2  # Example value for the number of classes per task
num_tasks = 10  # Example value for the number of tasks
num_agents = 10  # Example value for the number of agents


## CIFAR-100
num_classes = 100 # Example value for the number of classes
num_classes_per_task = 5  # Example value for the number of classes per task
num_tasks = 20 # Example value for the number of tasks
num_agents = 10  # Example value for the number of agents

In [ ]:
def sample_first_agent_tasks(num_tasks, num_classes, num_classes_per_task):
    tasks = []
    all_classes = set(range(num_classes))
    for _ in range(num_tasks):
        task = np.random.choice(list(all_classes), num_classes_per_task, replace=False)
        tasks.append(set(task))
    return tasks

def construct_new_task(ref_task, num_common_classes, num_classes_per_task, all_classes):
    # Determine the number of distinct classes to be added to the new task
    num_distinct_classes = num_classes_per_task - num_common_classes

    # Select 'num_common_classes' classes from the 'ref_task'
    common_classes = np.random.choice(list(ref_task), num_common_classes, replace=False)
    
    
    # Find the available classes to select distinct classes from (excluding the ones in 'ref_task')
    available_classes = list(all_classes - set(ref_task))
    
    # Select 'num_distinct_classes' distinct classes
    distinct_classes = np.random.choice(available_classes, num_distinct_classes, replace=False)
    
    # Construct and return the new task
    new_task = list(common_classes) + list(distinct_classes)
    np.random.shuffle(new_task)  # Shuffle the classes in the new task
    return new_task


def sample_subsequent_tasks(alpha, existing_tasks, num_classes, num_classes_per_task, num_tasks):
    new_tasks = []
    all_classes = set(range(num_classes))
    for _ in range(num_tasks):
        # TODO: potential bug here at flat_existing_tasks would 
        # amplify this effect, making this very "sharp". Slighly large `alpha` would
        # lead to a lot of duplicated tasks. Should probably remove the duplicated tasks
        # in `flat_existing_tasks` before sampling.
        flat_existing_tasks = [task for tasks in existing_tasks for task in tasks]
                # Remove duplicated tasks to avoid "sharp" effects
        unique_existing_tasks = [set(x) for x in set(tuple(x) for x in flat_existing_tasks)]

        print('flat_existing_tasks', flat_existing_tasks)
        print('unique_existing_tasks', unique_existing_tasks)
        # Now, select a reference task
        ref_task = np.random.choice(flat_existing_tasks)

        # Define possible IoU values and compute softmax probabilities
        possible_iou_values = np.array([i/(2* num_classes_per_task - i) for i in range(num_classes_per_task+1)])
        probs = softmax(possible_iou_values / alpha)
        
        # Select an IoU value based on the softmax probabilities
        num_common_classes = np.random.choice(range(len(possible_iou_values)), p=probs)
        
        # Construct a new task with the selected IoU value
        new_task = construct_new_task(ref_task, num_common_classes, num_classes_per_task, all_classes)

        print('ref_task', ref_task, 'new_task', new_task, possible_iou_values, probs, num_common_classes)
        
        new_tasks.append(set(new_task))
    print("No. of unique tasks:", len(unique_existing_tasks))
    return new_tasks



first_agent_tasks = sample_first_agent_tasks(num_tasks, num_classes, num_classes_per_task)
all_tasks = [first_agent_tasks]

for agent in range(1, num_agents):
    subsequent_tasks = sample_subsequent_tasks(alpha, all_tasks, num_classes, num_classes_per_task, num_tasks)
    all_tasks.append(subsequent_tasks)

In [ ]:
pprint(all_tasks)

In [ ]:
def calculate_iou(task1, task2):
    # Convert tasks to sets if they are not
    task1 = set(task1)
    task2 = set(task2)
    intersection = len(task1.intersection(task2))
    union = len(task1.union(task2))
    return intersection / union if union != 0 else 0

def calculate_average_iou(all_tasks):
    total_iou = 0
    total_pairs = 0
    for i in range(len(all_tasks)):
        for j in range(i+1, len(all_tasks)):  # Avoid comparing with itself and avoid duplicate pairs
            for task1 in all_tasks[i]:
                for task2 in all_tasks[j]:
                    # print(calculate_iou(task1, task2))
                    total_iou += calculate_iou(task1, task2)
                    total_pairs += 1
    return total_iou / total_pairs if total_pairs != 0 else 0

# Calculate the average pairwise task similarity
average_iou = calculate_average_iou(all_tasks)
print("Average Pairwise Task Similarity (IoU):", average_iou)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_tasks(all_tasks):
    plt.figure(figsize=(20, 5))
    num_agents = len(all_tasks)
    
    for agent_id, agent_tasks in enumerate(all_tasks):
        left = 0  # Initialize the left boundary of the bar
        for task_id, task in enumerate(agent_tasks):
            for class_id in task:
                color = "C"+ str(class_id)
                plt.barh(agent_id, 1, left=left, color=color)  # Draw the bar for the class
                left += 1  # Update the left boundary for the next class
    
    # Corrected task boundaries calculation
    task_boundaries = np.arange(num_classes_per_task, num_tasks * num_classes_per_task, num_classes_per_task)
    for boundary in task_boundaries:
        plt.axvline(x=boundary, color='k', linestyle='--')  # Add vertical lines to separate tasks

    plt.yticks(range(num_agents), labels=[f'Agent {i}' for i in range(num_agents)])
    plt.xlabel('Tasks over time')
    plt.ylabel('Agents')
    # plt.tight_layout()
    plt.show()

# Run the function with the task sequences
plot_tasks(all_tasks)

In [ ]:
# alphas = np.linspace(0.1, 10.0, 10)  # Replace with the alpha values you want to evaluate
# MNIST
# alphas = np.logspace(-1, 1, 10)  # Replace with the alpha values you want to evaluate
# CIFAR
alphas = np.logspace(-2, 1, 20)  # Replace with the alpha values you want to evaluate
# alphas = np.logspace(-2, 1, 10)  # Generate 10 alpha values between 0.1 (10^(-1)) and 1.0 (10^0) on a log scale
print(alphas)
num_samples = 10  # Number of samples for each alpha

In [ ]:

def calculate_avg_iou(tasks):
    ious = []
    for task1 in tasks:
        for task2 in tasks:
            if task1 is not task2:
                intersection = len(set(task1) & set(task2))
                union = len(set(task1) | set(task2))
                ious.append(intersection / union)
    return np.mean(ious)

mean_iou = []
std_iou = []

for alpha in alphas:
    iou_samples = []
    for _ in range(num_samples):
        first_agent_tasks = sample_first_agent_tasks(num_tasks, num_classes, num_classes_per_task)
        all_tasks = [first_agent_tasks]
        for agent in range(1, num_agents):
            subsequent_tasks = sample_subsequent_tasks(alpha, all_tasks, num_classes, num_classes_per_task, num_tasks)
            all_tasks.append(subsequent_tasks)
        
        # Flatten the all_tasks list and calculate the average IoU
        flat_tasks = [task for tasks in all_tasks for task in tasks]
        avg_iou = calculate_avg_iou(flat_tasks)
        iou_samples.append(avg_iou)
    
    mean_iou.append(np.mean(iou_samples))
    std_iou.append(np.std(iou_samples))



In [ ]:
# Plotting
plt.figure(figsize=(10,6))
plt.plot(alphas, mean_iou, label='Mean IoU')
plt.scatter(alphas, mean_iou, c='red')
plt.fill_between(alphas, np.array(mean_iou) - np.array(std_iou), np.array(mean_iou) + np.array(std_iou), color='b', alpha=0.2)
plt.xlabel('Alpha')
plt.ylabel('Average Pairwise Task Similarity (IoU)')
plt.title('Task Similarity vs. Alpha')
plt.legend()
plt.grid(True)

In [ ]:
alphas

In [ ]:
mean_iou